In [ ]:
import os
import glob
import math

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from shapely.geometry import Point, Polygon

In [ ]:
# define global variables 
img_path = "../data/01_raw/images/"
newpath = '../data/02_primary/images/quartered/true/'
falsepath = '../data/02_primary/images/quartered/false/'
slice_size = 1280

In [ ]:
img_list = glob.glob('../../data_raw/images/' + "*.jpg")
df_res = pd.DataFrame(columns=["image_id", "geometry", "class", "bounds","width", "height"])

# tile all images in a loop
for imname in img_list:
    im = Image.open(imname)
    imr = np.array(im, dtype=np.uint8)
    height = imr.shape[0]
    width = imr.shape[1]

    labels = df[df['image_id'] == imname.split('/')[-1]]
    boxes = []
    # convert bounding boxes to shapely polygons. We need to invert Y and find polygon vertices from center points
    for index, row in labels.iterrows():
        
        x1, y1, x2, y2 = row["bounds"][0], row["bounds"][1], row["bounds"][2], row["bounds"][3]
        
       # x1 = row[1]['x1'] - row[1]['w']/2
       # y1 = (height - row[1]['y1']) - row[1]['h']/2
       # x2 = row[1]['x1'] + row[1]['w']/2
       # y2 = (height - row[1]['y1']) + row[1]['h']/2
        boxes.append((row['class'], Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])))
    
    counter = 0
    print('Image:', imname)
    # create tiles and find intersection with bounding boxes for each tile
    for i in range((height // slice_size)):
        for j in range((width // slice_size)):
            x1 = (j*slice_size)
            y1 = (i*slice_size)
            x2 = ((j+1)*slice_size) - 1
            y2 = ((i+1)*slice_size) - 1
                        
            pol = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
            
            imsaved = False
            slice_labels = []
            slice_anno = []

            for box in boxes:
                if pol.intersects(box[1]):
                    inter = pol.intersection(box[1])   
                    if not imsaved:
                        sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                        sliced_im = Image.fromarray(sliced)
                        filename = imname.split('/')[-1]
                        slice_path = newpath + filename.replace('.jpg', f'_{i}_{j}.jpg')
                        
                        slice_labels_path = newpath + filename.replace('.jpg', f'_{i}_{j}.txt')
                        
                        print(slice_path)
                        sliced_im.save(slice_path)
                        imsaved = True                    
                    
                    # get the smallest polygon (with sides parallel to the coordinate axes) that contains the intersection
                    new_box = inter.envelope 
                    # get central point for the new bounding box 
                    centre = new_box.centroid
                    # get coordinates of polygon vertices
                    x, y = new_box.exterior.coords.xy
                    
                    
                    # if the bbox changed due to the image split, this means the airplane is truncated so the correct class will be set
                    objClass = str(box[0])
                    if(box[1].exterior.coords.xy != new_box.exterior.coords.xy):
                        objClass = "Truncated_airplane"
                        
                    # get bounding box width and height normalized to slice size
                    #new_width = (max(x) - min(x)) / slice_size
                    #new_height = (max(y) - min(y)) / slice_size
                    
                    
                    
                    # we have to normalize central x and invert y for yolo format
                    #new_x = (centre.coords.xy[0][0] - x1) / slice_size
                    #new_y = (y1 - centre.coords.xy[1][0]) / slice_size
                    
                    counter += 1

                    #slice_labels.append([box[0], new_x, new_y, new_width, new_height])
                    
                    
                    #if(min(x) > box[1][1][0]  )
                    
                    
                    
                    min_x, max_x, min_y, max_y = math.ceil(min(x) - (j*slice_size)), math.ceil(max(x)- (j*slice_size)), math.ceil(min(y)- (i*slice_size)), math.ceil(max(y)- (i*slice_size))
                    slice_width, slice_height = max_x - min_x, max_y - min_y 
                    geometry = [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]
                    bounds= (min_x, min_y, max_x, max_y)
                    
                    slice_anno.append([imname.split('/')[-1].replace('.jpg', f'_{i}_{j}.jpg'), geometry, objClass, bounds, slice_width, slice_height])
            
                    
            
            # save txt with labels for the current tile
            if len(slice_anno) > 0:
                #slice_df = pd.DataFrame(slice_labels, columns=['class', 'x1', 'y1', 'w', 'h'])
                #slice_df.to_csv(slice_labels_path, sep=' ', index=False, header=False, float_format='%.6f')
                
                df_anno = pd.DataFrame(slice_anno, columns=["image_id", "geometry", "class", "bounds","width", "height"])
                df_res = df_res.append(df_anno)
            
            # if there are no bounding boxes intersect current tile, save this tile to a separate folder 
            if not imsaved:
                sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                sliced_im = Image.fromarray(sliced)
                filename = imname.split('/')[-1]
                slice_path = falsepath + filename.replace('.jpg', f'_{i}_{j}.jpg')                

                sliced_im.save(slice_path)
                print('Slice without boxes saved')
                imsaved = True
df = df_res.reset_index(drop=True) 
df.to_csv('../sliced_images/quartered/annotaion/annotation.csv', sep=';', index=False)

In [ ]:
df[df['image_id'] == '074737f4-7f59-4729-be5d-67f6f1d34668_1_0.jpg']

In [ ]:
import os.path
import PIL.ImageDraw

imList = ['../sliced_images/quartered/true/074737f4-7f59-4729-be5d-67f6f1d34668_0_0.jpg', 
          '../sliced_images/quartered/true/074737f4-7f59-4729-be5d-67f6f1d34668_1_0.jpg', '../sliced_images/quartered/true/074737f4-7f59-4729-be5d-67f6f1d34668_1_1.jpg']

for pickone in imList:
    
    img = PIL.Image.open(pickone)
    draw = PIL.ImageDraw.Draw(img)

    for k, row in df[df['image_id'] == os.path.basename(pickone)].iterrows():
        draw.polygon(row['geometry'], outline=(255,0,0))
        draw.text(row['geometry'][0], row['class'], fill=(255,0,0))

    display(img)

## 2. Train-Test Split
---
run and save bounding boxes on training images and save training and test images seperate

In [ ]:
createBoundingBoxesOnImages = 1


from sklearn.model_selection import GroupShuffleSplit

X = df[["image_id", "geometry", "bounds", "width", "height"]]
y = df[['class']]

gs = GroupShuffleSplit(n_splits=2, test_size=.2, random_state=42)
train_ix, test_ix = next(gs.split(X, y, groups=df.image_id))

X_train = X.loc[train_ix]
y_train = y.loc[train_ix]

X_test = X.loc[test_ix]
y_test = y.loc[test_ix]


In [ ]:
if(createBoundingBoxesOnImages == 1):
    for id in X_train["image_id"].unique():
        pickone = '../sliced_images/quartered/true/' + id
        img = PIL.Image.open(pickone)
        #draw = PIL.ImageDraw.Draw(img)
        #for k, row in df[df['image_id'] == id].iterrows():
        #    draw.polygon(row['geometry'], outline=(255,0,0))
        #    draw.text(row['geometry'][0], row['class'], fill=(255,0,0))
        
        img.save('../images/train/' + id)

In [ ]:
if(createBoundingBoxesOnImages == 1):
    for id in X_test["image_id"].unique():
        pickone = '../sliced_images/quartered/true/' + id
        img = PIL.Image.open(pickone)
        
        img.save('../images/test/' + id)

In [ ]:
img_list = list('../images/train/*.jpg')
print("Number of Train-images: {}".format(len(img_list)))

## 3. Create XML Files
---

In [ ]:
import numpy as np
from pathlib import Path
import xml.etree.cElementTree as ET
from PIL import Image


def create_labimg_xml(image_path, df):

    temp_df = df
    image_path = Path(image_path)
    img = np.array(Image.open(image_path).convert('RGB'))

    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = str(image_path.parent.name)
    ET.SubElement(annotation, 'filename').text = str(image_path.name)
    ET.SubElement(annotation, 'path').text = str(image_path)

    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'

    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str (img.shape[1])
    ET.SubElement(size, 'height').text = str(img.shape[0])
    ET.SubElement(size, 'depth').text = str(img.shape[2])

    ET.SubElement(annotation, 'segmented').text = '0'

    for index, row in temp_df.iterrows():
        xmin, ymin, xmax, ymax = row["bounds"][0], row["bounds"][1], row["bounds"][2], row["bounds"][3]

        object = ET.SubElement(annotation, 'object')
        ET.SubElement(object, 'pose').text = 'Unspecified'
        if(row["class"] == "Truncated_airplane"):
            ET.SubElement(object, 'name').text = "Airplane"
            ET.SubElement(object, 'truncated').text = '1'
        elif(row["class"] == "Airplane"):
            ET.SubElement(object, 'name').text = "Airplane"
            ET.SubElement(object, 'truncated').text = '0'
        else:
            print("Error")
        ET.SubElement(object, 'difficult').text = '0'

        bndbox = ET.SubElement(object, 'bndbox')
        ET.SubElement(bndbox, 'xmin').text = str(xmin)
        ET.SubElement(bndbox, 'ymin').text = str(ymin)
        ET.SubElement(bndbox, 'xmax').text = str(xmax)
        ET.SubElement(bndbox, 'ymax').text = str(ymax)

    tree = ET.ElementTree(annotation)
    xml_file_name = image_path.parent / (image_path.name.split('.')[0]+'.xml')
    tree.write(xml_file_name)


# --------------------------------------------------------------------------------
# a quadrilateral bounding box(8 points) coordinate example
    

In [ ]:
for id in X_train["image_id"].unique():
        image_id = '../images/train/' + id
        df_temp = df[df['image_id'] == id]
        create_labimg_xml(image_id, df_temp)
        
for id in X_test["image_id"].unique():
        image_id = '../images/test/' + id
        df_temp = df[df['image_id'] == id]
        create_labimg_xml(image_id, df_temp)

# Validation
nohup python model_main_tf2.py --model_dir=models/my_efficientdet_d7_coco17_tpu-32/v1 --pipeline_config_path=models/my_efficientdet_d7_coco17_tpu-32/v1/pipeline.config --checkpoint_dir=models/my_efficientdet_d7_coco17_tpu-32/v1 &

## 4. Convert XML to Tensorflow Record
---

### Create train data:
python generate_tfrecord.py -x /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/images/train -l /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt -o /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/annotations/train.record

### Create test data:
python generate_tfrecord.py -x /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/images/test -l /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt -o /Users/lig/git/schule/airbus_aircraft_etection/tensorflow_env/TensorFlow/workspace/training_demo/annotations/test.record




python generate_tfrecord.py -x /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/images/test -l /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt -o /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/annotations/test.record

python generate_tfrecord.py -x /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/images/train -l /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt -o /home/zengerle@ab.ba.ba-ravensburg.de/airplane_detection/TensorFlow/workspace/training_demo/annotations/train.record


## 5. Train the Model
---
### Navigate to airplane_detection/TensorFlow/workspace/training_demo:

#### Default Training-Script:
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn/v1 --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/v1/pipeline.config &

#### Run Training in Background with Terminal Outputstream
nohup python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn/v1 --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/v1/pipeline.config & \
(Access Stream with: tail -f nohup.out)

#### Run Training in Background and save stream to log-file:
nohup python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn/v1 --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/v1/pipeline.config --alsologtostderr > /log/my_ssd_resnet50_v1_fpn/v1/train.log 2>&1 & \
(Access Log-File with: tail -f /log/model_main_tf2.log)


#### Note:
wenn die Batch-Size drastisch verringert wird (z.B. 128 -> 2 = Faktor x64) muss auch die Learning-Rate dementsprechend angepasst werden (ansonsten divergiert Loss-Function -> Division durch 0 -> Loss von NaN)

## 6. Evaluate the trained Modell on latest Checkpoints
---
### Navigate to airplane_detection/TensorFlow/workspace/training_demo:


##### Evaluate with the latest Model-Checkpoint:
nohup python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn/v1 --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/v1/pipeline.config --checkpoint_dir=models/my_ssd_resnet50_v1_fpn/v1 &

#### Note:
--checkpoint_dir will only execute the evaluation -> no training \
Run parallel / at the same time to the Training e.g. in new terminal to evaluate all the checkpoints over the whole trainings process


## 7. Additional Features:
---
### Navigate to airplane_detection/TensorFlow/workspace/training_demo:


#### Tensorboard - Visualize Training / Evaluation:
Start Tensorboard: \
tensorboard --logdir=models/my_ssd_resnet50_v1_fpn/v1/ &

#### Get nohup-stream:
tail -f nohup.out

#### Get GPU-Info:
nvidia-smi \
watch -n 1 nvidia-smi // live update every second

#### Set available GPUs (GPUs which should be used for the program):
export CUDA_VISIBLE_DEVICES=0 // set device by single indizes \
export CUDA_VISIBLE_DEVICES=3,4,5 // set list of devices \
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 // all devices 

#### Get Task-Manager
top

#### List servers
jobs -l

#### Kill process:
kill "pid"

### TO-DO

#### Hyperparametertuning
Parameter anschauen, in voller Auflösung trainieren 1280x1280

#### Eval-Bilder Predicten und anschauen 

#### Modell-Ergebnisse an UseCase anpassen
Ergebniss bewerten, auf UseCase beziehen, was uns das nun bringt, die Flugzeuge zu erkennen (Anzahl an predicteten Flugzeugen, ...)

#### Ergebnisse visualisieren
Count der Flugzeuge in e.g. Datafrage zu image-id \
Histogram - verteilung der Anzahl an Flugzeugen \
Durchschnitt an Flugzeugen pro Flughafen etc.


#### truncated / abgeschnittene Flugzeuge
auf slices predicten und danach vier slices wieder zu einem Bild zusammenfügen

#### Test - Train Split ändern (z.B. 90 - 10 für mehr Trainingsbilder)

#### Ausblick wie man Ergebnis verbessern kann
mehr Trainingsbilder (Data Augmentation) -> Spiegeln, verzerren, etc.

navigate to (training_demo)

nohup python model_main_tf2.py --model_dir=models/conf_ssd_resnet50_v1_fpn --pipeline_config_path=models/conf_ssd_resnet50_v1_fpn/pipeline.config &


(sollte sehr genau sein dauert aber lange)
nohup python model_main_tf2.py --model_dir=models/my_centernet_hg104_1024x1024_coco17_tpu-32 --pipeline_config_path=models/my_centernet_hg104_1024x1024_coco17_tpu-32/pipeline.config &


(sollte sehr schnell sein ist aber nicht so genau)
nohup python model_main_tf2.py --model_dir=models/my_centernet_mobilenetv2_fpn_od --pipeline_config_path=models/my_centernet_mobilenetv2_fpn_od/pipeline.config &


(sollte sehr genau sein dauert aber lange)
nohup python model_main_tf2.py --model_dir=models/my_efficientdet_d7_coco17_tpu-32 --pipeline_config_path=models/my_efficientdet_d7_coco17_tpu-32/pipeline.config &